In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)


In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [4]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [7]:
print(instrumentLookup(instrument_df,'RELIANCE'))

738561


In [8]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [9]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [10]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481


# Using ta library to compute macd

In [11]:
df = infy_df.copy()

In [12]:
df['fast_ma'] = ta.trend.EMAIndicator(close=df['close'], window=12).ema_indicator()
df['slow_ma'] = ta.trend.EMAIndicator(close=df['close'], window=26).ema_indicator()
df['macd'] = df['fast_ma'] - df['slow_ma']
df

,open,high,low,close,volume,fast_ma,slow_ma,macd
date,,,,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646,NaN,NaN,NaN
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481,NaN,NaN,NaN
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983,NaN,NaN,NaN
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608,NaN,NaN,NaN
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-05 12:00:00+05:30,1434.00,1435.00,1433.05,1434.70,44698,1430.849351,1425.598304,5.251047
2024-06-05 12:05:00+05:30,1434.75,1437.55,1434.00,1435.55,79640,1431.572528,1426.335467,5.237061
2024-06-05 12:10:00+05:30,1435.70,1437.50,1434.90,1437.40,39978,1432.469062,1427.155062,5.314000


# Manually computing macd

In [13]:
man_df = infy_df.copy()
man_df.head() 

,open,high,low,close,volume
date,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481


In [14]:
def compute_ema(df,n,column_name):
    df['sma'] = df['close'].rolling(n).mean()
    multiplier = 2 / (n + 1)
    df.dropna(inplace=True)
    for i in range(1,len(df)):
        if i == 1:
            df.loc[df.index[i],column_name] = df.loc[df.index[i - 1],'sma']
        else:
            df.loc[df.index[i],column_name] = (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],column_name]) * multiplier + df.loc[df.index[i - 1],column_name]
    return df

In [15]:
man_df = compute_ema(man_df,12,'fast')
man_df = compute_ema(man_df,26,'slow')
man_df['macd'] = man_df['fast'] - man_df['slow']
man_df

,open,high,low,close,volume,sma,fast,slow,macd
date,,,,,,,,,
2024-05-31 12:15:00+05:30,1425.60,1430.00,1425.25,1429.60,240778,1417.361538,1423.510969,NaN,NaN
2024-05-31 12:20:00+05:30,1429.95,1431.75,1428.40,1430.00,235171,1418.375000,1424.509282,1417.361538,7.147743
2024-05-31 12:25:00+05:30,1430.50,1431.35,1428.65,1430.35,240939,1419.292308,1425.407854,1418.323647,7.084207
2024-05-31 12:30:00+05:30,1430.00,1433.75,1429.70,1433.00,263825,1420.394231,1426.575876,1419.410784,7.165092
2024-05-31 12:35:00+05:30,1433.15,1433.50,1431.50,1432.40,241099,1421.401923,1427.471895,1420.372948,7.098947
...,...,...,...,...,...,...,...,...,...
2024-06-05 12:00:00+05:30,1434.00,1435.00,1433.05,1434.70,44698,1426.961538,1430.849351,1425.598304,5.251047
2024-06-05 12:05:00+05:30,1434.75,1437.55,1434.00,1435.55,79640,1428.013462,1431.572528,1426.335467,5.237061
2024-06-05 12:10:00+05:30,1435.70,1437.50,1434.90,1437.40,39978,1428.892308,1432.469062,1427.155062,5.314000


In [16]:
def compute_signal(df,n):
    df['signal_sma'] = df['macd'].rolling(n).mean()
    multiplier = 2 / (n + 1)
    df.dropna(inplace=True)
    for i in range(1,len(df)):
        if i == 1:
            df.loc[df.index[i],'signal_ma'] = df.loc[df.index[i - 1],'signal_sma']
        else:
            df.loc[df.index[i],'signal_ma'] = (df.loc[df.index[i],'macd'] - df.loc[df.index[i - 1],'signal_ma']) * multiplier + df.loc[df.index[i - 1],'signal_ma']
    return df

In [17]:
macd_df = compute_signal(man_df,9)
macd_df

,open,high,low,close,volume,sma,fast,slow,macd,signal_sma,signal_ma
date,,,,,,,,,,,
2024-05-31 13:00:00+05:30,1427.60,1428.95,1425.10,1426.95,226153,1424.755769,1427.901818,1422.919148,4.982670,6.439879,NaN
2024-05-31 13:05:00+05:30,1426.95,1428.65,1424.75,1428.50,226510,1425.359615,1427.993846,1423.332544,4.661302,6.163608,6.439879
2024-05-31 13:10:00+05:30,1428.70,1430.10,1427.55,1427.55,198419,1425.880769,1427.925562,1423.644948,4.280614,5.852097,6.008026
2024-05-31 13:15:00+05:30,1427.40,1429.00,1426.20,1428.75,196000,1426.213462,1428.052398,1424.023100,4.029298,5.503676,5.612280
2024-05-31 13:20:00+05:30,1428.80,1429.55,1424.10,1426.70,352697,1426.538462,1427.844337,1424.221389,3.622948,5.117454,5.214414
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05 12:00:00+05:30,1434.00,1435.00,1433.05,1434.70,44698,1426.961538,1430.849351,1425.598304,5.251047,6.305570,6.141218
2024-06-05 12:05:00+05:30,1434.75,1437.55,1434.00,1435.55,79640,1428.013462,1431.572528,1426.335467,5.237061,6.039210,5.960387
2024-06-05 12:10:00+05:30,1435.70,1437.50,1434.90,1437.40,39978,1428.892308,1432.469062,1427.155062,5.314000,5.809564,5.831109


In [18]:
macd_df.to_csv('macd_infy.csv')